In [1]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader
import torch
from torchvision import models
import torch.nn as nn
import os
from torch.utils.data import Dataset
import cv2
import glob
import torch
from torchvision.transforms import transforms
from collections import defaultdict
from tqdm import tqdm

In [10]:
import matplotlib.pyplot as plt

In [2]:
label_dict = {"dekopon" : 0, "grapefruit":1, "kenpei":2, "orange":3}

class custom_dataset(Dataset):
    def __init__(self, image_file_path, transform=None):
        self.image_file_paths = glob.glob(os.path.join(image_file_path, "*", "*.png"))
        self.transform = transform

    def __getitem__(self, index):
        # image loader
        image_path = self.image_file_paths[index]
        image = cv2.imread(image_path)

        # cv2 -> BGR -> RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # label
        label_temp = os.path.basename(image_path)
        label_temp = label_temp.split("_")
        label_temp = label_temp[0]
        label = label_dict[label_temp]

        if self.transform is not None:
            image = self.transform(image=image)["image"]

        image= image.float()        

        return image, label

    def __len__(self):
        return len(self.image_file_paths)


In [3]:
class hy_params:
  lr = 0.001
  batch_size = 8
  epoch = 50
  num_classes =4

In [4]:

# save model
def save_model(model, save_dir, file_name="last.pt"):
    os.makedirs(save_dir, exist_ok=True)
    output_path = os.path.join(save_dir, file_name)
    if isinstance(model, nn.DataParallel):
        print("멀티 GPU 저장 !!")
        torch.save(model.module.state_dict(), output_path)

    else:
        print("싱글 GPU 저장 !!")
        torch.save(model.state_dict(), output_path)


# 새로운 train loop
def train(number_epoch, train_loader, val_loader, criterion, optimizer, model, save_dir, device):
    print("start training...")
    running_loss = 0.0
    total = 0
    best_loss = 77777

    for epoch in range(number_epoch):
        for i, (images, labels) in tqdm(enumerate(train_loader)):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            _, argmax = torch.max(outputs, 1)
            acc = (labels ==argmax).float().mean()
            total += labels.size(0)

            if ( i + 1 ) % 5 == 0:
                print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.4f}".format(
                    epoch + 1,
                    number_epoch,
                    i+1,
                    len(train_loader),
                    loss.item(),
                    acc.item()*100,
                ))
        
        avg_loss, val_acc = validate(epoch, model, val_loader, criterion, device)

        # 특정 epoch마다 저장하고 싶은 경우
        if epoch % 10 ==0:
            save_model(model, save_dir, file_name=f"{epoch}.pt")

        # best save
        if val_acc > best_loss:
            print(f"best save >>> {epoch}")
            best_loss = val_acc
            save_model(model, save_dir, file_name="best.pt")

    # last save
    save_model(model, save_dir, file_name="final.pt")


def validate(epoch, model, val_loader, criterion, device):
    print("start validation...")
    with torch.no_grad():
        model.eval()
        total = 0
        correct = 0
        total_loss = 0
        cnt=0
        batch_loss = 0

        for i, (images, labels) in tqdm(enumerate(val_loader)):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            batch_loss += loss.item()

            total += labels.size(0)
            _, argmax = torch.max(outputs, 1)
            correct += (argmax ==labels).sum().item()
            total_loss += loss.item()
            cnt += 1

    avg_loss = total_loss /cnt
    val_acc = (correct/total*100)

    print("val # {} acc {:.2f}% avg loss {:.4f}".format(
        epoch + 1,
        correct/total *100,
        avg_loss,
    ))

    plt.plot(epochs, loss.itme(),'b-', label='Train Loss')
    plt.plot(epochs, total_loss,'b-', label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    
    plt.plot(epochs, val_acc,'b-', label='Validation Acc')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    model.train()

    return avg_loss, val_acc


In [6]:

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train Aug
train_transform = A.Compose([
    A.Resize(height=224,width=224),
    ToTensorV2()
])

# Val Aug
val_transform = A.Compose([
    A.Resize(height=224,width=224),
    ToTensorV2()
])

# dataset
train_dataset = custom_dataset("./data/train", transform=train_transform)
val_dataset = custom_dataset("./data/val", transform=val_transform)

# dataloader
train_loader = DataLoader(train_dataset, batch_size=hy_params.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=hy_params.batch_size, shuffle=False)

# # model call
# net = models.__dict__["resnet18"](pretrained=False, num_classes=hy_params.num_classes)
# # print(net)
# pretrained = True, num_classes =4 수정방법
net = models.__dict__["resnet18"](pretrained=True)
net.fc = nn.Linear(512,4)
# print(net)
net.to(device)

# criterion
criterion = nn.CrossEntropyLoss()

# optimizer
optim = torch.optim.Adam(net.parameters(), lr=hy_params.lr)

# model save dir
model_save_dir="./model_save"
os.makedirs(model_save_dir, exist_ok=True)

# # 새로운 train
# # def train(number_epoch, train_loader, val_loader, criterion, optimizer, model, save_dir, device):
# history = train(
#                 number_epoch=hy_params.epoch,
#                 train_loader=train_loader, 
#                 val_loader=val_loader,
#                 criterion=criterion,
#                 optimizer=optim,
#                 model=net,
#                 save_dir=model_save_dir,
#                 device=device
#                 )




In [12]:
# train
# def train(number_epoch, train_loader, val_loader, criterion, optimizer, model, save_dir, device):
train(number_epoch=hy_params.epoch,
    train_loader=train_loader, 
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optim,
    model=net,
    save_dir=model_save_dir,
    device=device)
    # validate(val_loader, net, criterion, epochs, device)


start training...


5it [00:11,  2.28s/it]

Epoch [1/50], Step [5/25], Loss: 0.0003, Acc: 100.0000


10it [00:22,  2.29s/it]

Epoch [1/50], Step [10/25], Loss: 0.0306, Acc: 100.0000


...

Epoch [50/50], Step [20/25], Loss: 0.0012, Acc: 100.0000


25it [00:55,  2.21s/it]


Epoch [50/50], Step [25/25], Loss: 0.0008, Acc: 100.0000
start validation...


3it [00:02,  1.43it/s]

val # 50 acc 91.67% avg loss 0.3591
싱글 GPU 저장 !!


In [ ]:
import matplotlib.pyplot as plt

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss,'b-', label='Validation Loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
for epochs in range(1, hy_params.epoch+1):    
    

In [ ]:

def acc_function(correct, total):
    acc = correct /total * 100
    return acc

def test(model, test_loader, device):

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad() :
        for batch_idx, (data, target, path) in enumerate(test_loader):
            image_path = path[0]
            img = cv2.imread(image_path)
            
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, argmax = torch.max(output, 1)
            total == target.size(0)
            correct += (target ==argmax).sum().item()
            font_italic = "FONT_ITALIC"

            argmax_temp = argmax.item()
            cv2.putText(img, str(argmax_temp), (90,90), cv2.FONT_ITALIC, 1, (0,0,0), 2)
            cv2.imshow("Test", img)
            cv2.waitKey(0)            

        acc=acc_function(correct, total)
        print("acc for {} image : {:.2f}%".format(total, acc))
        # print("Testing Accuracy: {:.2%}".format(acc))

def main() :
    test_transform = A.Compose([
        A.Resize(height=224,width=224),
        ToTensorV2()
    ])

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = models.__dict__["resnet18"](pretrained=False, num_classes=4)
    net = net.to(device)

    net.load_state_dict(torch.load("./model_save/final.pt", map_location=device))
    test_data = custom_dataset("./data/val", transform=test_transform)

    test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

    # 데이터을 잘 받아오는지 확인
    # for i in test_data:
    #     print(i)
    # for i,j in test_loader:
    #     print(i,j)



# 디버깅
if __name__ =="__main__":
    main()

    



In [ ]:


class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()
    
    def reset(self):
        self.metrics = defaultdict(lambda : {"val":0, "count":0, "avg":0})

    def update(self, metrics_name, val):
        metrics = self.metrics[metrics_name]

        metrics["val"] += val
        metrics["count"] += 1
        metrics["avg"] = metrics["val"] / metrics["count"]

    def __str__(self):
        return " | ".join([
            "{metric_name} : {avg.{float_precision}f}".format(
                metric_name = metric_name, avg=metrics["avg"],
                float_precision=self.float_precision
            )
            for(metric_name, metrics) in self.metrics.items()
        ])

In [ ]:
# # train
# for epochs in range(1, hy_params.epoch+1):
#     train(train_loader, net, criterion, optim, epochs, device)
#     validate(val_loader, net, criterion, epochs, device)